<a href="https://colab.research.google.com/github/krithiyer/safe-elections/blob/main/CS294S_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### Approach

Use Quora Question Pairs fine-tuned BERT (sequence classification) model to select top 3 question to question matches from covid-related-questions. (User query to top-3 question match)

Use SQUAD fine-tune BERT Question and Answer model to get the answer to the user question.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/cs294s/

/content/drive/My Drive/cs294s


In [ ]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
pip install transformers

     |████████████████████████████████| 1.3MB 8.0MB/s 
     |████████████████████████████████| 2.9MB 30.6MB/s 
     |████████████████████████████████| 890kB 40.3MB/s 
     |████████████████████████████████| 1.1MB 54.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1ce3428190783d30c04ba5819c2fcb6ce7fa6a6c02758237c938ea4ab240f2c8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# The name of the folder containing the model files.
input_dir = './models_saved_qqp_oct9Final/'

# Load our fine-tuned model, and configure it to return the "hidden states", 
# from which we will be taking our text embeddings.
# num_labels = 2,  is this needed while loading saved models
model = BertForSequenceClassification.from_pretrained(
    input_dir,
    output_hidden_states = True, # Whether the model returns all hidden-states.
) 

# Load the tokenizer. we have not saved it.
#tokenizer = BertTokenizer.from_pretrained(output_dir)

model.cuda()

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 256  # if need be increase to 512

#user_query = "will i get covid if I vote"
#faq_question = "will i get covid if I vote"

def create_tokens (user_query, faq_question):
  input_ids = tokenizer.encode(
                        user_query, faq_question,                   # Sentence to encode.
                        add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN,       # Truncate all sentences. 
                        truncation = True                       
                   )  

  results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long", 
                              truncating="post", padding="post")
    
    # Remove the outer list.
  input_ids = results[0]

    # Create attention masks    
  attn_mask = [int(i>0) for i in input_ids]
    
    # Cast to tensors.
  input_ids = torch.tensor(input_ids)
  attn_mask = torch.tensor(attn_mask)

    # Add an extra dimension for the "batch" (even though there is only one 
    # input in this batch.)
  input_ids = input_ids.unsqueeze(0)
  attn_mask = attn_mask.unsqueeze(0)  

  return input_ids, attn_mask

load the database of Covid and Election related questions

In [ ]:
import pandas
election_faq_df = pandas.read_excel('./final_database.xlsx')
election_faq_df.head()
proc_data_df = election_faq_df.dropna()

Compare user query with all database questions


In [ ]:
from keras.preprocessing.sequence import pad_sequences

user_query = "May election officials require the use of a pencil with an eraser to mark on the electronic voting system?"

master_match_df = pandas.DataFrame(columns = ['User_Question', 'FAQ_Question', 'FAQ_Answer', 'Logit_0', 'Logit_1', 'Probability_0', 'Probability_1'])
torch.no_grad()
for row in proc_data_df.itertuples():
  inpt_id, att_mask = create_tokens(user_query, row.Question)
  inpt_id = inpt_id.to(device)
  att_mask = att_mask.to(device)
  with torch.no_grad():
    logits, encoded_layers = model(input_ids = inpt_id, token_type_ids = None, attention_mask = att_mask)
  probabilities = torch.nn.functional.softmax(logits, dim=1)
  sclr_logits = logits.cpu().numpy()
  sclr_prob = probabilities.cpu().numpy()
  temp_df = pandas.DataFrame( [[ user_query, row.Question, row.Answer, sclr_logits[0][0], sclr_logits[0][1], sclr_prob[0][0], sclr_prob[0][1] ] ],
                             columns = ['User_Question', 'FAQ_Question', 'FAQ_Answer', 'Logit_0', 'Logit_1', 'Probability_0', 'Probability_1'])

  master_match_df = master_match_df.append(temp_df)

find the top 3 match

In [ ]:
sorted_qa_df = master_match_df.sort_values(by=['Probability_1'], ascending=False)

In [ ]:
sorted_qa_df.head()

,User_Question,FAQ_Question,FAQ_Answer,Logit_0,Logit_1,Probability_0,Probability_1
0,May election officials require the use of a pe...,May election officials require the use of a pe...,"Yes. However, you should contact your vendor t...",1.560913,-0.916088,0.922514,0.077486
0,May election officials require the use of a pe...,What are election officials doing to protect v...,The SEC and county election officials are taki...,4.857038,-4.366715,0.999901,0.000099
0,May election officials require the use of a pe...,May election officials place social distancing...,"No, you cannot place social distancing require...",4.920997,-4.422826,0.999913,0.000087
0,May election officials require the use of a pe...,Can election officials require voters to wear ...,"No, you cannot require voters to wear face cov...",4.961453,-4.494057,0.999922,0.000078
0,May election officials require the use of a pe...,May election officials require an assistant/in...,"No, just as you cannot require a voter to use ...",5.002065,-4.525341,0.999927,0.000073


In [ ]:
num_selected_ans = 3
top_answer_choices = []
for i in range(num_selected_ans):
  top_answer_choices.append(sorted_qa_df['FAQ_Answer'].iloc[i])

In [ ]:
top_answer_choices

['Yes. However, you should contact your vendor to make sure that it will work on the type of electronic voting machine being used. In addition, if you impose such a requirement, you must provide the pencils (or other stylus alternatives) for voters to utilize when voting.',
 'The SEC and county election officials are taking steps to make polling places as safe as possible for voters and poll managers: Managers will receive special training on sanitizing surfaces and applying social distancing concepts. Managers will be equipped with masks, face shields and gloves. Managers will be provided with sanitizing wipes and will regularly clean common surfaces throughout the day. Check-in stations and voting equipment will be spaced to keep voters and managers at least six feet apart. Hand sanitizer will be provided for voters and managers. Voters will be provided with a cotton swab for making selections on the touchscreen. Voters are asked to wear their own mask if they have one and to bring t

###Part 2: Answer Matching


In [ ]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering")

num_selected_ans has the number of answer choices

user_query has the question

top_answer_choices has the list of possible answers.

In [ ]:
for j in range(num_selected_ans):
  result = qa_pipeline(question=user_query, context=top_answer_choices[j])
  print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


Answer: 'if you impose such a requirement, you must provide the pencils', score: 0.0201, start: 142, end: 204


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


Answer: 'Voters are asked to wear their own mask', score: 0.0117, start: 652, end: 691


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


Answer: 'poll watcher.', score: 0.0339, start: 57, end: 70


File name  TestSet_CS294W.xlsx  User_Question, Matched_Question

Take questions from TestSet_CS294W  (query)

Get the top 5 matching questions.

Put the query and answers (for the top 4 matched questions) through Q&A model

In [ ]:
testSet_q_df = pandas.read_excel('./TestSet_CS294W.xlsx')
testSet_q_df.head()
proc_test_q_df = testSet_q_df.dropna()

In [ ]:
proc_test_q_df.head()

,User_Question,Matched_Question
0,Can I still register to vote in Texas?,When is the deadline to register to vote in Texas
1,Will my polling location practice safety preca...,How should election officials sanitize equipme...
2,Should I wear a mask to the location?,Do I have to wear a face mask to vote?
3,Can I be socially distant in my polling location?,May election officials place social distancing...
4,What should I bring to the polling location?,May election workers give voters pencils as th...


Find the top five matching questions - this is user_question to faq_question matching using the fine-tuned sequence classifiction model. 

In [ ]:
num_selected_ans = 5

master_qQA_df = pandas.DataFrame(columns = ['User_Question', 'FAQ_Question', 'FAQ_Answer','Logit_0', 'Logit_1', 'Probability_0', 'Probability_1'])
for row in proc_test_q_df.itertuples():
  tempResults_eachQ_df = pandas.DataFrame(columns = ['User_Question', 'FAQ_Question', 'FAQ_Answer','Logit_0', 'Logit_1', 'Probability_0', 'Probability_1'])
  for faq_row in proc_data_df.itertuples():
    inpt_id, att_mask = create_tokens(row.User_Question, faq_row.Question)
    inpt_id = inpt_id.to(device)
    att_mask = att_mask.to(device)
    with torch.no_grad():
      logits, encoded_layers = model(input_ids = inpt_id, token_type_ids = None, attention_mask = att_mask)
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    sclr_logits = logits.cpu().numpy()
    sclr_prob = probabilities.cpu().numpy()
    temp_df = pandas.DataFrame( [[ row.User_Question, faq_row.Question, faq_row.Answer, sclr_logits[0][0], sclr_logits[0][1], sclr_prob[0][0], sclr_prob[0][1] ] ],
                             columns = ['User_Question', 'FAQ_Question', 'FAQ_Answer','Logit_0', 'Logit_1', 'Probability_0', 'Probability_1'])
    tempResults_eachQ_df = tempResults_eachQ_df.append(temp_df)
    
  sorted_tempResults_df = tempResults_eachQ_df.sort_values(by=['Probability_1'], ascending=False)
  master_qQA_df = master_qQA_df.append(sorted_tempResults_df.head(num_selected_ans))
    # orig question, match_question, match_answer
master_qQA_df.to_excel('./results_qQA_match.xlsx')

In [ ]:
master_qQA_df.head(10)

,User_Question,FAQ_Question,FAQ_Answer,Logit_0,Logit_1,Probability_0,Probability_1
0,Can I still register to vote in Texas?,Can I still register to vote in Texas,The deadline for registering to vote in Texas ...,0.283392,0.205086,0.519566,0.480434
0,Can I still register to vote in Texas?,Can I still register to vote in Arkansas,The deadline for registering to vote in Arkans...,2.114738,-1.573490,0.975594,0.024406
0,Can I still register to vote in Texas?,Can I still register to vote in Oklahoma,The deadline for registering to vote in Oklaho...,2.118939,-1.618785,0.976745,0.023255
0,Can I still register to vote in Texas?,Can I still register to vote in Mississippi,The deadline for registering to vote in Missis...,2.150707,-1.606130,0.977176,0.022824
0,Can I still register to vote in Texas?,Can I still register to vote in Montana,The deadline for registering to vote in Montan...,2.112983,-1.651839,0.977353,0.022647
0,Will my polling location practice safety preca...,What are election officials doing to protect v...,The SEC and county election officials are taki...,4.994951,-4.525153,0.999927,0.000073
0,Will my polling location practice safety preca...,Will polling places be open on Election Day? W...,"Yes, polling places will be open on Election D...",4.988162,-4.548024,0.999928,0.000072
0,Will my polling location practice safety preca...,What security measures are in place to ensure ...,Multiple security measures are in place to kee...,5.011796,-4.552426,0.999930,0.000070
0,Will my polling location practice safety preca...,How do I find out if my polling place has moved?,Contact your local Circuit Clerk’s Office or E...,5.045072,-4.563527,0.999933,0.000067
0,Will my polling location practice safety preca...,How do I know that my mail ballot is safe to o...,The CDC says that coronavirus transmission fro...,5.034688,-4.590800,0.999934,0.000066


Match user_query to faq answer and get the match scores.

Using the FAQ answers (for the top 5 matched questions) match the query to answers using question answer pipeline

In [ ]:
master_final_match_df = pandas.DataFrame(columns = ['User_Question', 'FAQ_Question', 'FAQ_Answer','Matched_Answer', 'Match_Score'])
for row in master_qQA_df.itertuples():
  result = qa_pipeline(question=row.User_Question, context=row.FAQ_Answer)
  temp_df = pandas.DataFrame( [[ row.User_Question, row.FAQ_Question, row.FAQ_Answer, result['answer'], round(result['score'], 4) ] ],
                             columns = ['User_Question', 'FAQ_Question', 'FAQ_Answer','Matched_Answer', 'Match_Score'])
  master_final_match_df = master_final_match_df.append(temp_df)
master_final_match_df.to_excel('./results_qA_mat.xlsx')

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/transformers/tokenization

In [ ]:
master_final_match_df.head(10)

,User_Question,FAQ_Question,FAQ_Answer,Matched_Answer,Match_Score
0,Can I still register to vote in Texas?,Can I still register to vote in Texas,The deadline for registering to vote in Texas ...,2020-10-05,0.4866
0,Can I still register to vote in Texas?,Can I still register to vote in Arkansas,The deadline for registering to vote in Arkans...,2020-10-05,0.4326
0,Can I still register to vote in Texas?,Can I still register to vote in Oklahoma,The deadline for registering to vote in Oklaho...,2020-10-09,0.5162
0,Can I still register to vote in Texas?,Can I still register to vote in Mississippi,The deadline for registering to vote in Missis...,The deadline for registering to vote in Missis...,0.3653
0,Can I still register to vote in Texas?,Can I still register to vote in Montana,The deadline for registering to vote in Montan...,2020-10-26,0.5660
0,Will my polling location practice safety preca...,What are election officials doing to protect v...,The SEC and county election officials are taki...,Managers will receive special training on sani...,0.0453
0,Will my polling location practice safety preca...,Will polling places be open on Election Day? W...,"Yes, polling places will be open on Election D...",We have relocated 50 polling places in Minneap...,0.0396
0,Will my polling location practice safety preca...,What security measures are in place to ensure ...,Multiple security measures are in place to kee...,Multiple security measures,0.0735
0,Will my polling location practice safety preca...,How do I find out if my polling place has moved?,Contact your local Circuit Clerk’s Office or E...,confirm your polling place,0.1161
0,Will my polling location practice safety preca...,How do I know that my mail ballot is safe to o...,The CDC says that coronavirus transmission fro...,washing your hands after collecting mail from ...,0.1126


Model based version (not using the pipeline)

Ignore this section - the model section is kept for possible use in google cloud based system.

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", return_dict=True)


In [ ]:
text = r"""
... 🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
... architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
... Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
... TensorFlow 2.0 and PyTorch.
... """

questions = [
...     "How many pretrained models are available in 🤗 Transformers?",
...     "What does 🤗 Transformers provide?",
...     "🤗 Transformers provides interoperability between which frameworks?",
... ]



In [ ]:
for question in questions:
...     inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
...     input_ids = inputs["input_ids"].tolist()[0]
...
...     text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        model_outputs = model(**inputs)
...     answer_start_scores = model_outputs.start_logits
        answer_end_scores = model_outputs.end_logits
...
...     answer_start = torch.argmax(
...         answer_start_scores
...     )  # Get the most likely beginning of answer with the argmax of the score
...     answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
...
...     answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
...
...     print(f"Question: {question}")
...     print(f"Answer: {answer}")

TypeError: ignored